# Creating Vector Database

In [ ]:
# If we run this machine on GPU machine it will be faster.

In [2]:
import pandas as pd

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

apipath = r'H:\\My Drive\\config\\hbqa.txt'
import configparser

config = configparser.ConfigParser()
config.read(apipath)
secret_key = config['global']['OPENAI_API_KEY']
datapath = config['global']['DATA_FOLDER']
corpuspath = config['global']['CORPUS_FOLDER']
PINECONE_API_KEY = config['global']['PINECONE_API_KEY']
PINECONE_ENV = config['global']['PINECONE_ENV']

# PINECONE is Vector Database. To store the vector so that we can quickly search the vector space.
# https://app.pinecone.io
# get PINECONE_API_KEY key from app.pinecone.io
# find your PINECONE_ENVIRONMENT next to the api key in pinecone console

# Creating Embedding for Chunks

In [7]:
# This chunk was created to use with ChatGPT Front End to create prompt for generating QA.
# At this point of time there was no clarity around chunk size, number of question, types of question, \
# as per initial investigation cost of ChatGPT API was very high. So intially this Chunk was created. \
# Unfortuntely the main file is lost. But the prompt created from those chunks still exists. So, after\
#  reverse engineering, created Chunk from the Prompt text.
df_hbqa_for_fe = pd.read_csv(datapath+"05-QA_Final_ChatGPT_FE-Created-14Sep23.csv")
print(df_hbqa_for_fe.shape)
df_hbqa_for_fe.head(4)

(256, 7)


,Context_Len,Len,Section_Id,Chunk_Id,Prompt,Chunk,QA
0,2000,2030,Book01_046,193,"Write 1 question, corresponding answer, and th...","Sauti continued, 'The Muni, having said so unt...",\nQ1: Why did the Muni wander over the earth a...
1,2000,3462,Book01_104,425,"Write 6 question, corresponding answer, and th...",In this connection there is another old histor...,Q1: Who was the wise Rishi mentioned in the te...
2,2000,1974,Book01_007,83,"Write 1 question, corresponding answer, and th...","Sauti said, 'the god of fire enraged at the cu...",Q1: What did Agni say was the consequence of a...
3,2000,1492,Book01_005,78,"Write 1 question, corresponding answer, and th...",And the Rakshasa saw the apartment in which th...,Q1: What did the Rakshasa ask the god of fire ...


In [8]:
# There are 256 chunks

In [10]:
#This chunk was created to use with ChatGPT API to create prompt for QA. 
df_hbqa_for_api = pd.read_csv(datapath+"03-Chunked_Book.csv")
print(df_hbqa_for_api.shape)
df_hbqa_for_api.head(4)

(1963, 7)


,Section_Id,Chunk_Id,Chunk,Chunk_Letters,Chunk_Words,Chunk_Approx_Tokens,Approx_Ques
0,Book01_002,1,THE MAHABHARATA ADI PARVA Section I\nOm! Havin...,7511,1214,1615.0,4
1,Book01_002,2,The Rishi Vyasa published this mass of knowled...,7551,1261,1677.0,5
2,Book01_002,3,"Vyasa executed the compilation of the Bharata,...",6986,1162,1545.0,4
3,Book01_002,5,'When I heard that Yudhishthira had been follo...,10832,1949,2592.0,7


In [11]:
# There are 1963 chunks

In [12]:
# Both chunk files were created with different chapters of Mahabharat Book, different chunk size, they are created 
# at different time therefore chunk_id in 
# the both the files will be duplicate. To avoid that problem we are manually changing chunk_id
df_hbqa_for_fe['Chunk_Id'] = df_hbqa_for_fe['Chunk_Id']+10000

In [15]:
df_mahabharat_chunk = pd.concat([df_hbqa_for_api[['Chunk_Id','Chunk']],
df_hbqa_for_fe[['Chunk_Id','Chunk']]])

In [18]:
print(df_mahabharat_chunk.shape)
df_mahabharat_chunk.sample(5)


(2219, 2)


,Chunk_Id,Chunk
1780,2567,"and diamonds, as also with an altar constructe..."
1321,1977,"Section CI\n""Yudhishthira said, 'Of what dispo..."
186,10277,"Vaisampayana said, 'O king, they are as follow..."
1755,2537,"Section XCIII\n ""Yudhishthira said, 'If Brahm..."
886,1362,"Section LXXVIII\n""Sanjaya said, 'Hearing these..."


In [21]:
df_mahabharat_chunk.to_csv(datapath+'08.1-Mahabharat_Chunk.csv')

In [19]:
# !pip install -U sentence-transformers

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

c:\Users\hari_\anaconda3\envs\hbqa\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%time
Chunk_Sentences = df_mahabharat_chunk.Chunk.tolist()
Chunk_Embeddings = model.encode(Chunk_Sentences)

In [11]:
import torch

In [13]:
DEVICE='gpu'

In [14]:
ChunkVectors= torch.tensor(Chunk_Embeddings, dtype=torch.float).to(DEVICE)

In [15]:
# Flatten the tensors into 1D arrays
ChunkVec_list = ChunkVectors.tolist()

In [ ]:
df_mahabharat_chunk['ChunkVector'] = ChunkVectors.tolist()


In [ ]:
!pip install pinecone-client

In [ ]:
import pinecone      

pinecone.init(      
	api_key='ddf2ef12-9108-4e80-b438-8f86a0b617c0',      
	environment='asia-southeast1-gcp-free'      
)      
index = pinecone.Index('HBQA')

In [19]:
import os
import pinecone

# # get api key from app.pinecone.io
# PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
# # find your environment next to the api key in pinecone console
# PINECONE_ENV = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENV
)

In [21]:
# index_name = 'semantic-search-fast'
index_name = "HBQA"

In [29]:
!pip uninstall pinecone_datasets

^C


In [27]:
from pinecone_datasets import load_dataset

dataset = load_dataset('quora_all-MiniLM-L6-bm25')
# # we drop sparse_values as they are not needed for this example
# dataset.documents.drop(['metadata'], axis=1, inplace=True)
# dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
# # we will use 80K rows of the dataset between rows 240K -> 320K
# dataset.documents.drop(dataset.documents.index[320_000:], inplace=True)
# dataset.documents.drop(dataset.documents.index[:240_000], inplace=True)
# dataset.head()

ModuleNotFoundError: No module named 'pinecone_datasets'

In [ ]:
import time

# only create index if it doesn't exist
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=len(dataset.documents.iloc[0]['values']),
        metric='cosine'
    )
    # wait a moment for the index to be fully initialized
    time.sleep(1)

# now connect to the index
index = pinecone.GRPCIndex(index_name)